# Training a basic SAE with SAELens

This tutorial demonstrates training a simple, relatively small Sparse Autoencoder, specifically on the tiny-stories-1L-21M model. 

As the SAELens library is under active development, please open an issue if this tutorial is stale [here](https://github.com/jbloomAus/SAELens).

## Setup

In [ ]:
try:
    # import google.colab # type: ignore
    # from google.colab import output
    %pip install sae-lens transformer-lens
except:
    from IPython import get_ipython  # type: ignore

    ipython = get_ipython()
    assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

In [ ]:
import torch
import os

from sae_lens import (
    SAE,
    ActivationsStore,
    HookedSAETransformer,
    LanguageModelSAERunnerConfig,
    SAEConfig,
    SAETrainingRunner,
    upload_saes_to_huggingface,
)

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")
wandb.login(key=secret_value_0)

In [ ]:
import huggingface_hub
huggingface_api = ""
huggingface_hub.login(token=huggingface_api)

In [ ]:
import torch
import os
from sae_lens import LanguageModelSAERunnerConfig, SAETrainingRunner, upload_saes_to_huggingface

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def train(layer, l1_coef, lr, upload = True):
    total_training_steps = 20_000
    batch_size = 4096
    total_training_tokens = total_training_steps * batch_size

    lr_warm_up_steps = 0
    lr_decay_steps = total_training_steps // 5
    l1_warm_up_steps = total_training_steps // 20

    cfg = LanguageModelSAERunnerConfig(
        model_name="tiny-stories-1L-21M",
        hook_name="blocks." + str(layer) + ".hook_mlp_out",
        hook_layer=layer,
        d_in=1024,
        dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",
        is_dataset_tokenized=True,
        streaming=True,
        mse_loss_normalization=None,
        expansion_factor=8,
        b_dec_init_method="zeros",
        apply_b_dec_to_input=False,
        normalize_sae_decoder=False,
        scale_sparsity_penalty_by_decoder_norm=True,
        decoder_heuristic_init=True,
        init_encoder_as_decoder_transpose=True,
        normalize_activations="expected_average_only_in",
        lr=lr,
        adam_beta1=0.9,
        adam_beta2=0.999,
        lr_scheduler_name="constant",
        lr_warm_up_steps=lr_warm_up_steps,
        lr_decay_steps=lr_decay_steps,
        l1_coefficient=l1_coef,
        l1_warm_up_steps=l1_warm_up_steps,
        lp_norm=1.0,
        train_batch_size_tokens=batch_size,
        context_size=256,
        n_batches_in_buffer=64,
        training_tokens=total_training_tokens,
        store_batch_size_prompts=16,
        use_ghost_grads=False,
        feature_sampling_window=1000,
        dead_feature_window=1000,
        dead_feature_threshold=1e-4,
        log_to_wandb=True,
        wandb_project="tiny_stories_saes",
        wandb_log_frequency=30,
        eval_every_n_wandb_logs=20,
        device=device,
        seed=42,
        n_checkpoints=1,
        checkpoint_path="checkpoints",
        dtype="float32",
    )

    # Start training
    sparse_autoencoder = SAETrainingRunner(cfg).run()
    if upload:
        path = os.path.join(cfg.checkpoint_path, os.listdir(cfg.checkpoint_path)[0])
        sae_dict = {
            str(cfg.l1_coefficient): path
        }
        upload_saes_to_huggingface(
            saes_dict=sae_dict,
            hf_repo_id="anhtu77/sae-tiny-stories-1L-21M",
        )
        print(f"Uploaded SAE for layer {layer} to Hugging Face Hub.")
    del sparse_autoencoder

def main():
    # Train the model with different layers
    layers = [0]
    l1_coefs = [8, 16]
    lrs = [5e-5]
    for l1_coef in l1_coefs:
        for lr in lrs:
            for layer in layers:
                print(f"Training layer {layer}...")
                train(layer, l1_coef, lr)
                print(f"Finished training layer {layer}.")

main()